<a href="https://colab.research.google.com/github/TheDampChris/hahtables/blob/master/Homework_3_Comp462.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install gffpandas
import gffpandas.gffpandas as gffpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 178 kB 14.7 MB/s 
  Created wheel for gffpandas: filename=gffpandas-1.2.0-py2.py3-none-any.whl size=6263 sha256=bf6e7fcfc6a2e776b6facd1f1bd7842c512c87c48e137d2af50c22333c14cc86
  Stored in directory: /root/.cache/pip/wheels/a9/d0/2a/cb5ebf14451176d87e738547d58cb2ec39ebc46b19649133cc
Successfully built gffpandas


In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 13.7 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
import math

In [ ]:
annotation = gffpd.read_gff3('Vibrio_cholerae.GFC_11.37.gff3')
filtered_df = annotation.filter_feature_of_type(['CDS'])

In [ ]:
cond = filtered_df.df['strand'] == "+" # consider backward strands as intragenic
cond_rev = filtered_df.df['strand'] == "-"
genic_df = filtered_df.df.where(cond)
genic_df.dropna(inplace=True);
intragenic_df = filtered_df.df.where(cond_rev)
intragenic_df.dropna(inplace=True)
genic_df['length'] = genic_df['end'] - genic_df['start']
intragenic_df['length'] = intragenic_df['end'] - intragenic_df['start']
Average_genic_region = genic_df['length'].sum()/genic_df.shape[0]

genic_df['start'] = genic_df['start'].astype(int)
genic_df['end'] = genic_df['end'].astype(int)

intragenic_df['start'] = intragenic_df['start'].astype(int)
intragenic_df['end'] = intragenic_df['end'].astype(int)


In [ ]:
from Bio import SeqIO
with open('Vibrio_cholerae.GFC_11.dna.nonchromosomal.fa') as fasta_file:  # Will close handle cleanly
    identifiers = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(seq_record.seq)



In [ ]:
seq_genic = {} #create a dictionary

for row in genic_df.itertuples():
  if row.seq_id in seq_genic:
    seq_genic[row.seq_id].append([row.start, row.end])
  else:
    seq_genic[row.seq_id] = [[row.start, row.end]]

In [ ]:
seq_intragenic = {}
for key in seq_genic:
  for i in range(0, len(seq_genic[key])):
    if i == 0:
      if key in seq_intragenic:
        seq_intragenic[key].append([1, (seq_genic[key][0][0] - 1)])
      else:
        seq_intragenic[key] = [[1, (seq_genic[key][0][0] - 1)]]
    elif not i + 1 == len(seq_genic[key]):
      if seq_genic[key][i - 1][1] == seq_genic[key][i][0]:
        continue
      else:
        seq_intragenic[key].append([(seq_genic[key][i - 1][1] + 1), (seq_genic[key][i][0] - 1)])
    elif i + 1 == len(seq_genic[key]):
      seq_intragenic[key].append([(seq_genic[key][i][1] + 1), len(sequence[int(key[-3 : ]) - 1])])

In [ ]:
genic_arr = []

for key in seq_genic:
  for i in range(0, len(seq_genic[key])):
      id = int(key[-3 : ])-1
      start = (seq_genic[key][i][0] - 1)
      end = (seq_genic[key][i][1] - 1)
      genic_arr.append(sequence[id][start : end])


intragenic_arr = []

for key in seq_intragenic:
  for i in range(0, len(seq_intragenic[key])):
      intragenic_arr.append(sequence[int(key[-3 : ]) - 1][(seq_intragenic[key][i][0] - 1) : (seq_intragenic[key][i][1] - 1)])



Average Length of intragenic regions

In [ ]:
average = 0
intragenic_arr = [x for x in intragenic_arr if x != '']

for seq in intragenic_arr:
  average += len(seq)
print(average/len(intragenic_arr))

1200.1271850512358


Average Length of genic regions

In [ ]:
average = 0
genic_arr = [x for x in genic_arr if x != '']

for seq in genic_arr:
  average += len(seq)
print(average/len(genic_arr))

990.4515103338633


Intragenic Nucleotide Frequency

In [ ]:
# Genic nucleotide frequency

frequency_nucleo = {'A' : 0, 'T' : 0, 'G' : 0, 'C': 0}
nucleotides = 0
for seq in intragenic_arr:
  nucleotides += len(seq)
  for nucleo in seq:
    if nucleo == 'A':
      frequency_nucleo['A'] = frequency_nucleo.get('A') + 1
    elif nucleo == 'T':
      frequency_nucleo['T'] = frequency_nucleo.get('T') + 1
    elif nucleo == 'G':
      frequency_nucleo['G'] = frequency_nucleo.get('G') + 1
    elif nucleo == 'C':
      frequency_nucleo['C'] = frequency_nucleo.get('C') + 1
    else:
      continue

for nucleo in frequency_nucleo:
  frequency_nucleo[nucleo] = frequency_nucleo.get(nucleo) / nucleotides

print(frequency_nucleo)

{'A': 0.2648056690796786, 'T': 0.26453143654153594, 'G': 0.22660949638148659, 'C': 0.24405339799729886}


Codon Frequency Table

In [ ]:
codon_table = {}
nucleotides = ['A', 'T', 'G', 'C']

for pos1 in nucleotides:
  for pos2 in nucleotides:
    for pos3 in nucleotides:
      codon = pos1+pos2+pos3
      codon_table[codon] = 0

for seq in genic_arr:
  if len(seq) >= 6: 
    for i in range(3, int(len(seq)/3), 3):
      codon_table[seq[i : i+3]] = codon_table.get(seq[i : i+3]) + 1
  else:
    continue

frequency = 0
for value in codon_table:
  frequency += codon_table[value]

for value in codon_table:
  codon_table[value] = codon_table.get(value)/frequency

print(codon_table)

{'AAA': 0.0362000834975776, 'AAT': 0.019311242075011892, 'AAG': 0.013427575560453217, 'AAC': 0.019651057797799937, 'ATA': 0.00388360826043477, 'ATT': 0.0317096614464499, 'ATG': 0.021612279969319494, 'ATC': 0.0252094721205472, 'AGA': 0.002490363797003796, 'AGT': 0.012160548365486374, 'AGG': 0.0008107032243657583, 'AGC': 0.014019825820169519, 'ACA': 0.0083546122702603, 'ACT': 0.013369321436546696, 'ACG': 0.011641115760653223, 'ACC': 0.021092847364486345, 'TAA': 4.854510325543462e-06, 'TAT': 0.016116974280804296, 'TAG': 9.709020651086924e-06, 'TAC': 0.013675155587055933, 'TTA': 0.020291853160771672, 'TTT': 0.02777265357243415, 'TTG': 0.02251521888987058, 'TTC': 0.012951833548549958, 'TGA': 2.9127061953260773e-05, 'TGT': 0.0056215229569793294, 'TGG': 0.013247958678408108, 'TGC': 0.004199151431595095, 'TCA': 0.010956629804751594, 'TCT': 0.012286765633950503, 'TCG': 0.009306096294066818, 'TCC': 0.006558443449809217, 'GAA': 0.03821470528267814, 'GAT': 0.036156392904647706, 'GAG': 0.0228113440

Start Codon Frequency Table

In [ ]:
Start_codon_table = {}
nucleotides = ['A', 'T', 'G', 'C']
for pos1 in nucleotides:
  for pos2 in nucleotides:
    for pos3 in nucleotides:
      codon = pos1+pos2+pos3
      Start_codon_table[codon] = 0

for preseq in genic_arr:
  if len(preseq) >= 3:
    Start_codon_table[preseq[0:3]] = Start_codon_table.get(preseq[0:3]) + 1

Start_frequency = 0
for value in Start_codon_table:
  Start_frequency += Start_codon_table[value]

for value in Start_codon_table:
  Start_codon_table[value] = Start_codon_table.get(value)/Start_frequency

print(Start_codon_table)

{'AAA': 0.0, 'AAT': 0.0005299417064122947, 'AAG': 0.0, 'AAC': 0.0, 'ATA': 0.0, 'ATT': 0.0, 'ATG': 0.8871224165341812, 'ATC': 0.0, 'AGA': 0.0, 'AGT': 0.0, 'AGG': 0.0, 'AGC': 0.0, 'ACA': 0.0, 'ACT': 0.0, 'ACG': 0.0, 'ACC': 0.0, 'TAA': 0.0, 'TAT': 0.0, 'TAG': 0.0, 'TAC': 0.0, 'TTA': 0.0, 'TTT': 0.0, 'TTG': 0.03762586115527292, 'TTC': 0.0, 'TGA': 0.0, 'TGT': 0.0, 'TGG': 0.0, 'TGC': 0.0, 'TCA': 0.0, 'TCT': 0.0, 'TCG': 0.0, 'TCC': 0.0, 'GAA': 0.0, 'GAT': 0.0, 'GAG': 0.0, 'GAC': 0.0, 'GTA': 0.0, 'GTT': 0.0, 'GTG': 0.07366189719130896, 'GTC': 0.0005299417064122947, 'GGA': 0.0, 'GGT': 0.0, 'GGG': 0.0, 'GGC': 0.0, 'GCA': 0.0, 'GCT': 0.0, 'GCG': 0.0, 'GCC': 0.0, 'CAA': 0.0, 'CAT': 0.0, 'CAG': 0.0, 'CAC': 0.0, 'CTA': 0.0, 'CTT': 0.0, 'CTG': 0.0, 'CTC': 0.0, 'CGA': 0.0, 'CGT': 0.0, 'CGG': 0.0, 'CGC': 0.0005299417064122947, 'CCA': 0.0, 'CCT': 0.0, 'CCG': 0.0, 'CCC': 0.0}


Stop Codon Frequency Table

In [ ]:
Stop_codon_table = {}
nucleotides = ['A', 'T', 'G', 'C']
for pos1 in nucleotides:
  for pos2 in nucleotides:
    for pos3 in nucleotides:
      codon = pos1+pos2+pos3
      Stop_codon_table[codon] = 0

for preseq in genic_arr:
  if len(preseq) >= 3:
    Stop_codon_table[preseq[-3 :]] = Stop_codon_table.get(preseq[-3 :]) + 1

Stop_frequency = 0
for value in Stop_codon_table:
  Stop_frequency += Stop_codon_table[value]

for value in Stop_codon_table:
  Stop_codon_table[value] = Stop_codon_table.get(value)/Stop_frequency

print(Stop_codon_table)

{'AAA': 0.0, 'AAT': 0.0, 'AAG': 0.0, 'AAC': 0.0, 'ATA': 0.2289348171701113, 'ATT': 0.0, 'ATG': 0.06412294647588765, 'ATC': 0.0, 'AGA': 0.0, 'AGT': 0.0, 'AGG': 0.0, 'AGC': 0.0005299417064122947, 'ACA': 0.0, 'ACT': 0.0, 'ACG': 0.0, 'ACC': 0.0, 'TAA': 0.0, 'TAT': 0.0, 'TAG': 0.0, 'TAC': 0.0, 'TTA': 0.20985691573926868, 'TTT': 0.0, 'TTG': 0.04345521992580816, 'TTC': 0.0, 'TGA': 0.0, 'TGT': 0.0, 'TGG': 0.0, 'TGC': 0.0, 'TCA': 0.0, 'TCT': 0.0, 'TCG': 0.0, 'TCC': 0.0, 'GAA': 0.0, 'GAT': 0.0, 'GAG': 0.0, 'GAC': 0.0, 'GTA': 0.19448860625331213, 'GTT': 0.0005299417064122947, 'GTG': 0.03815580286168521, 'GTC': 0.0, 'GGA': 0.0, 'GGT': 0.0, 'GGG': 0.0, 'GGC': 0.0, 'GCA': 0.0, 'GCT': 0.0, 'GCG': 0.0, 'GCC': 0.0, 'CAA': 0.0, 'CAT': 0.0, 'CAG': 0.0, 'CAC': 0.0, 'CTA': 0.17594064652888183, 'CTT': 0.0, 'CTG': 0.04345521992580816, 'CTC': 0.0005299417064122947, 'CGA': 0.0, 'CGT': 0.0, 'CGG': 0.0, 'CGC': 0.0, 'CCA': 0.0, 'CCT': 0.0, 'CCG': 0.0, 'CCC': 0.0}


Frequencies (USED IN VITERBI AS INPUT)

In [ ]:
states = ('Intergenic', 'Start', 'Genic', 'Stop')
start_p = {'Intergenic': 1, 'Start': 0, 'Genic': 0, 'Stop': 0}
trans_p = {
    'Intergenic': {'Intergenic': (1199/1200), 'Start': (1/1200), 'Genic': 0, 'Stop': 0},
    'Start': {'Intergenic': 0,'Start': 0, 'Genic': 1, 'Stop': 0},
    'Genic': {'Intergenic': 0, 'Start': 0, 'Genic': (329/330), 'Stop' : (1/330)},
    'Stop' : {'Intergenic': 1, 'Start': 0, 'Genic': 0, 'Stop': 0}
}
emis_p = {
    'Intergenic': frequency_nucleo,
    'Start': Start_codon_table,
    'Genic': codon_table,
    'Stop' : Stop_codon_table
}

Logarithm calculator function

In [ ]:
def logit(list):
  prob = 0
  for i in list:
    prob = math.log(i)
  return prob

Helper functions

In [ ]:
def finders(dict):
  answer_dict = {}
  for key in dict:
      ex = dict[key]
      first = True
      while not len(ex) == 0:
        try:
          if first:
            start = ex.index('Start')
            stop = ex.index('Stop') + 2
            first = False
          else:
            start = ex.index('Start', answer_dict.get(key)[-1][1])
            stop = ex.index('Stop', answer_dict.get(key)[-1][1] + 3) + 2
          if key not in answer_dict:
            answer_dict[key] = [[start, stop]]
          else:
            answer_dict[key].append([start, stop])
        except ValueError:
          ex = []
  return answer_dict

In [ ]:
def verbose(dict, filename):
  text = ''
  with open(filename, 'w') as f:
    for key in dict:
      for i in range(0, len(dict[key])):
        towrite = key + '\t' + 'ena' + '\t' + 'CDS' + '\t' + str(dict[key][i][0]) + ' ' + str(dict[key][i][1]) + '\t' + '.' + '\t' + '+' + '\t' + '0' + '\t' + '.' + '\t' +'\n'
        print
        text += towrite 
        f.write(towrite)
  print(text)
  f.close()

Viterbi Algorigthm that spits out a file

*   It is run by inputting a fasta file 'here'
*   states, start probabilities, transition probabilities and emissions probabilities were all computed beforehand
*   It will write to a file called 'filename'





In [ ]:
def viterbi_alg(here, states, start_p, trans_p, emis_p, filename):
  with open(here) as fasta_file:  # Will close handle cleanly
    sequence = {}
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
      sequence[seq_record.id] = seq_record.seq

  observ_dict = {}
  for observations in sequence:
    contig = observations
    observations = sequence[observations]

    t = 1
    skip = 0
    copyoverstate = ''
    statepath = ['Intergenic']
    statepath_prob = [(start_p['Intergenic'] * emis_p['Intergenic'][observations[0]])]
    while t < len(observations):
      if skip > 0:
        statepath_prob.append(statepath_prob[-1])
        statepath.append(statepath[-1])
        skip -= 1
        t += 1
        continue
      prev_st_selected = statepath[-1]
      max_state_prob = []
      for st in states:
        if st == 'Intergenic':
          max_prob = trans_p[prev_st_selected][st] * emis_p[st][observations[t]]
          if not max_prob == 0:
            max_prob = statepath_prob[- 1] +  logit([trans_p[prev_st_selected][st],  emis_p[st][observations[t]]])
          else:
            max_prob = -math.inf
        else:
          if t+2 >= len(observations):
            max_prob = -math.inf
          else:
            max_prob = trans_p[prev_st_selected][st] * emis_p[st][observations[t : t + 3]]
            if not max_prob == 0:
              max_prob = statepath_prob[- 1]  + logit([trans_p[prev_st_selected][st], emis_p[st][observations[t : t + 3]]])
            else:
              max_prob = -math.inf
            if st == 'Start':
              if trans_p[prev_st_selected]['Intergenic'] == 0 or trans_p[prev_st_selected]['Intergenic'] == 0:
                max_prob2 = -math.inf
              else:
                max_prob2 = statepath_prob[- 1] + logit([trans_p[prev_st_selected]['Intergenic'], emis_p['Intergenic'][observations[t]]])
                max_prob2 += logit([trans_p['Intergenic']['Intergenic'] * emis_p['Intergenic'][observations[t+1]]])
                max_prob2 += logit([trans_p['Intergenic']['Intergenic'] * emis_p['Intergenic'][observations[t+2]]])
              max_prob = max(max_prob, max_prob2)
  
        max_state_prob.append((max_prob))
      most_likely_state = states[max_state_prob.index(max(max_state_prob))]
      if not most_likely_state is 'Intergenic':
        skip = 2
      most_likely_prob = max(max_state_prob)
      statepath_prob.append(most_likely_prob)
      statepath.append(most_likely_state)
      t += 1
    observ_dict[contig] = statepath
  verbose(finders(observ_dict), filename)

In [ ]:
there = viterbi_alg('Vibrio_vulnificus.ASM74310v1.dna.nonchromosomal.fa', states, start_p, trans_p, emis_p, 'Vibrio_vulnificus.gff3')

contig_32	ena	CDS	64 102	.	+	0	.	
contig_32	ena	CDS	283 291	.	+	0	.	
contig_32	ena	CDS	423 443	.	+	0	.	
contig_32	ena	CDS	506 526	.	+	0	.	
contig_32	ena	CDS	572 640	.	+	0	.	
contig_32	ena	CDS	642 662	.	+	0	.	
contig_32	ena	CDS	687 695	.	+	0	.	
contig_32	ena	CDS	699 713	.	+	0	.	
contig_32	ena	CDS	753 761	.	+	0	.	
contig_32	ena	CDS	801 818	.	+	0	.	
contig_32	ena	CDS	860 889	.	+	0	.	
contig_32	ena	CDS	900 914	.	+	0	.	
contig_32	ena	CDS	938 1039	.	+	0	.	
contig_32	ena	CDS	1118 1168	.	+	0	.	
contig_32	ena	CDS	1181 1195	.	+	0	.	
contig_32	ena	CDS	1319 1327	.	+	0	.	
contig_32	ena	CDS	1344 1352	.	+	0	.	
contig_32	ena	CDS	1361 1381	.	+	0	.	
contig_32	ena	CDS	1391 1435	.	+	0	.	
contig_32	ena	CDS	1440 1448	.	+	0	.	
contig_32	ena	CDS	1521 1553	.	+	0	.	
contig_32	ena	CDS	1620 1655	.	+	0	.	
contig_32	ena	CDS	1707 1721	.	+	0	.	
contig_32	ena	CDS	1758 1778	.	+	0	.	
contig_32	ena	CDS	1824 1835	.	+	0	.	
contig_32	ena	CDS	1962 2027	.	+	0	.	
contig_32	ena	CDS	2063 2074	.	+	0	.	
contig_32	ena	CDS	2127 2141

Compare function

In [ ]:
def compare(gff3, gff4):
  annotation = gffpd.read_gff3(gff3)
  annotation2 = gffpd.read_gff3(gff4)
  filtered_df1 = annotation.filter_feature_of_type(['CDS'])
  filtered_df2 = annotation2.filter_feature_of_type(['CDS'])
  cond1 = filtered_df1.df['strand'] == "+" # consider backward strands as intragenic
  cond2 = filtered_df2.df['strand'] == "-"
  gff3 = filtered_df1.df.where(cond1)
  gff3.dropna(inplace=True);
  gff4 = filtered_df1.df.where(cond2)
  gff3.dropna(inplace=True);